Schuller et al., eIF5A Functions Globally in Translation Elongation and Termination, Molecular Cell (2017), http:// dx.doi.org/10.1016/j.molcel.2017.03.003

Polarity at position i in gene with length l is defined as:

$w_i = {{2i-(l+1)}\over{l - 1}}$

$p_i = {{d_i w_i}\over{\sum_{i=1}^l d_i}}$

In [1]:
import sys
import xlrd
import pandas as pd
import numpy as np
from numpy import median
import os
import fnmatch 
import plotly.graph_objects as go
import plotly.express as px

In [3]:
# Range of genes: length in bp
d1 = 0
d2 = 35000000
num_c = 10
m1 = 1200
m2 = 1500
viewer = 2000
exclude = 0
cwd = os.getcwd()
print(cwd)
# Ribosome counts directory
#Ribocsv_dir = '/home/jiz225/Zid-Lab/polarity/output/Glu_readd/EY-Ribo/'
Ribocsv_dir = cwd+'/output/Anna/'
# mRNA counts directory
RNAcsv_dir = cwd+'/output/Glu_readd/EY-RNA/'
# Delta Dhh1 file
dhh1file = cwd+'/Dhh1_Length_Short.xlsx'
# Mitochondia genes
mitofile = cwd+'/Williams_mito_Proxi_ribosom_Profiling.xls'

/Users/jiz225/Desktop/ZidLab/polarity


In [55]:
red = ['YLL026W', 'YDR258C', 'YPL240C', 'YBR169C', 'YER103W', 
       'YGR142W', 'YDR171W', 'YCR021C', 'YBR072W', 'YFL014W', 'YLR327C']
isred = []
df_dhh1_all = pd.read_excel(dhh1file, index_col=None)
for i in df_dhh1_all.get('Name'):
    if i in red:
        isred.append('y')
    else:
        isred.append('n')
df_dhh1_all['Type'] = isred
print(max(df_dhh1_all.iloc[:,3]), min(df_dhh1_all.iloc[:,3]))
df_dhh1 = df_dhh1_all[(df_dhh1_all.iloc[:,3] > d1) & (df_dhh1_all.iloc[:,3] < d2)]
print(df_dhh1.columns)

df_dhh1_ex = df_dhh1[(df_dhh1.iloc[:,-1] == 'n')]
print('number of genes excluding red gens',len(df_dhh1_ex))
df_dhh1_in = df_dhh1
print('number of genes including red gens',len(df_dhh1_in))

dhh1_top = df_dhh1.sort_values(by='Delta_Dhh1', ascending=False).iloc[:600,:]
#print(dhh1_top.iloc[:,6].quantile([0.05,0.25,0.5,0.75,0.995,1]))
dhh1_bot = df_dhh1.sort_values(by='Delta_Dhh1', ascending=True).iloc[:800,:]
#print(dhh1_bot.iloc[:,6].quantile([0.05,0.25,0.5,0.75,0.995,1]))

14733 201
Index(['Name', 'Code', 'Annotation', 'ORF Length', 'YEP(ip-total) -G Dhh1',
       'YPD(ip-total) +G Dhh1', 'Delta_Dhh1', 'LengthPredicted Dhh1',
       'D_predicted', 'tai', 'BY_Ribo_B_Polarity', 'BY_Ribo_A_Polarity',
       'Ribosome Occupancy +G Ribo/mRNA - 1P/1Tt', 'Ribo +G - 1Pd_C',
       'mRNA +G - 1Ttd-Ribozero', '-G/+G mRNA - 4Tdat/1Tdat',
       '-G/+G Ribosome occupancy', 'Type'],
      dtype='object')
number of genes excluding red gens 4310
number of genes including red gens 4321


In [56]:
df_mito = pd.read_excel(mitofile,index_col=None)
df_mito = df_mito.rename(columns={'#gene':'Name'})
df_mito = df_mito[(df_mito.Name.isin(df_dhh1.Name)) & (df_mito.mitop2 == 'yes')]  
df_nonmito = df_dhh1[(~df_dhh1.Name.isin(df_mito.Name))]
print('df_mito', df_mito.shape, df_mito.columns)
print('df_nonmito', df_nonmito.shape, df_nonmito.columns)
df_mito_diff = df_mito[(df_mito['minus.chx.enriched'] == 'no') & (df_mito['plus.chx.enriched'] == 'yes')]
print('df_mito_diff ', df_mito_diff.shape, max(df_mito_diff['aa.length'])*3,min(df_mito_diff['aa.length'])*3)
#df_mito_yes = df_mito[(df_mito['minus.chx.enriched'] == 'yes') & (df_mito['plus.chx.enriched'] == 'yes')]
df_mito_yes = df_mito[(df_mito['minus.chx.enriched'] == 'yes')]
print('df_mito_yes ', df_mito_yes.shape, max(df_mito_yes['aa.length'])*3,min(df_mito_yes['aa.length'])*3)
#df_mito_no = df_mito[(df_mito['minus.chx.enriched'] == 'no') & (df_mito['plus.chx.enriched'] == 'no')]
df_mito_no = df_mito[(df_mito['plus.chx.enriched'] == 'yes')]
print('df_mito_no ', df_mito_no.shape, max(df_mito_no['aa.length'])*3,min(df_mito_no['aa.length'])*3)

df_mito (519, 29) Index(['Name', 'common.name', 'aa.length', 'mitop2', 'mito.compartment',
       'mito.GO', 'gfp.localization', 'unannonated.enriched', 'mitoprot.score',
       'signalp.predicted', 'glycosylated', 'gene.ohnolog',
       'minus.chx.enriched', 'plus.chx.enriched', 'om45.2mchx.total.cts',
       'om45.2mchx.total.rpkm', 'om45.2mchx.pulldown.cts',
       'om45.2mchx.pulldown.rpkm', 'om45.2mchx.log2enrich',
       'om45.2m.total.cts', 'om45.2m.total.rpkm', 'om45.2m.pulldown.cts',
       'om45.2m.pulldown.rpkm', 'om45.2m.log2enrich', 'ssh1.2mchx.total.cts',
       'ssh1.2mchx.total.rpkm', 'ssh1.2mchx.pulldown.cts',
       'ssh1.2mchx.pulldown.rpkm', 'ssh1.2mchx.log2enrich'],
      dtype='object')
df_nonmito (3802, 18) Index(['Name', 'Code', 'Annotation', 'ORF Length', 'YEP(ip-total) -G Dhh1',
       'YPD(ip-total) +G Dhh1', 'Delta_Dhh1', 'LengthPredicted Dhh1',
       'D_predicted', 'tai', 'BY_Ribo_B_Polarity', 'BY_Ribo_A_Polarity',
       'Ribosome Occupancy +G Ribo/mRNA -

In [2]:
'''
Description: Parse the input file to start at 15 bps upstream of each gene
    Input: path for input file and path for new csv file generated
'''
def makecsv(inputpath, outputpath, ext):
    inputf = open(inputpath,'r')
    outputf = open(outputpath, 'w')
    outputf.write('Name'+ ',' + 'Orient' + ',' + 'Start' + ',' + 'End' + ',' + 'Position' + ',' +'Length' + ',' + 'RelativePosition' + ',' + 'Counts' + '\n')
    rposit = -1000    
    lines = inputf.readlines()
    for line in lines:
        string = line.rstrip() 
        cells = string.split('\t')
        name = cells[0][:7] 
        orient = cells[0][6:7]
        start = cells[1]
        end = cells[2]
        length = abs(int(start) - int(end))
        riboCount = cells[4]
        posit = cells[3]
        
        if orient == 'W':
            rposit = (int(cells[3])+1 - (int(start)))
    
        elif orient == "C":
            rposit = ((int(start))-(int(cells[3])-1))
            
        if rposit > 0 and rposit <(length-15):
            outputf.write(str(name) + ',' + str(orient) + ',' + str(start) + ',' + str(end) + ',' + str(posit) + ',' + str(length-15) + ',' + str(rposit) + ',' + str(riboCount) + '\n')
    inputf.close()
    outputf.close()

In [3]:
def calculatePolarity(i, l, densityati, density):
    normDist = (2*i - (l + 1))/(l - 1)   
    polarity = densityati*normDist/density
    
    return polarity

def PolarityPerGene(inputpath, destdir):
    df = pd.read_csv(inputpath)
    names = inputpath.split('.')
    names1 = names[0].split('/')
    
    #this is the final output file
    nameofcsv = destdir + names1[len(names1) - 1] + str("_polarity&gene.csv")
    print(nameofcsv)
    
    len_dict = dict(zip(df['Name'],df['Length']))
    den_dict = df.groupby('Name')['Counts'].agg('sum').to_dict()
    
    Polaritylist = []
    for i in range (len(df)):
        polarityofi = calculatePolarity(df.iloc[i,6], df.iloc[i,5], df.iloc[i,7], den_dict.get(df.iloc[i,0]))
        Polaritylist.append(polarityofi)
    df['Polarity'] = Polaritylist
    
    #calculate polarity per gene
    pol_dict = df.groupby('Name')['Polarity'].agg('sum').to_dict()
    
    Lengthlist = []
    Totalcountlist = []
    names = []
    genepol = []
    for name in pol_dict.keys():
        if name in len_dict.keys() and name in den_dict.keys():
            Lengthlist.append(len_dict.get(name))
            Totalcountlist.append(den_dict.get(name))
            names.append(name)
            genepol.append(pol_dict.get(name))
        
    d = {'Name':names,'Length':Lengthlist,'Counts':Totalcountlist,'PolarityPerGene': genepol}
    dfcsv = pd.DataFrame(data=d)
    dfcsv = dfcsv.dropna(how='any',axis=0)
    dfcsv.to_csv(nameofcsv)
   

In [6]:
inputlist = []
outputlist = []
inputfolder = os.listdir(inpath)
extension = "*.xls"
for doc in inputfolder:
    if fnmatch.fnmatch(doc, extension):
        #change this input and ouput file path
        inputpath = inpath + doc
        namelist = doc.split('.')
        name = namelist[0]
        outputpath = midpath + name + str(".csv")
        inputlist.append(inputpath)
        outputlist.append(outputpath)
print('\n Number of possible input files: ' + str(len(inputlist)))
print('\n Number of possible output csv files: ' + str(len(outputlist)))
    
for i in range(0, len(inputlist)):
    makecsv(inputlist[i], outputlist[i], extension)
    PolarityPerGene(outputlist[i], destdir)


 Number of possible input files: 5

 Number of possible output csv files: 5
/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/Glu_readd/EY-RNA/EY-RNA_logphase-2_4_TGACCA_polarity&gene.csv
/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/Glu_readd/EY-RNA/EY-RNA-Glu15+Glu1-2_7_CAGATC_polarity&gene.csv
/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/Glu_readd/EY-RNA/EY-RNA-Glu7_5-2_5_ACAGTG_polarity&gene.csv
/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/Glu_readd/EY-RNA/EY-RNA-Glu15-2_6_GCCAAT_polarity&gene.csv
/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/Glu_readd/EY-RNA/EY-RNA-Glu15+Glu5-1_1_ATCACG_polarity&gene.csv


In [57]:
csv_dir = destdir
os.chdir(csv_dir)
wd = os.getcwd()

csv_list=[]
tag = 'Ribo'
for root,dirs,files in os.walk(wd):
    for file in files:
        if file.split('.')[-1] == "csv" and tag in file:
            csv_list.append(csv_dir + file)
csv_list_s = sorted(csv_list)
print(len(csv_list_s))
print(csv_list_s)

5
['/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/Glu_readd/EY-Ribo/EY-Ribo-Glu15+Glu1-1_8_ACTTGA_polarity&gene.csv', '/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/Glu_readd/EY-Ribo/EY-Ribo-Glu15+Glu5-2_9_ACGACT_polarity&gene.csv', '/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/Glu_readd/EY-Ribo/EY-Ribo-Glu15-1_7_CAGATC_polarity&gene.csv', '/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/Glu_readd/EY-Ribo/EY-Ribo-Glu7_5-1_6_GCCAAT_polarity&gene.csv', '/Users/jiz225/Desktop/ZidLab/polarity/outputcsv/Glu_readd/EY-Ribo/EY-Ribologphase-1_5_ACAGTG_polarity&gene.csv']


In [67]:
keys = []
values_list = []
fit_list = []
times = []
num_c = 10
llist = []
for i in range(len(csv_list_s)):
    file = csv_list_s[i]
    time = file.split('/')[-1].split(tag)[1].split('_polarity&gene')[0]
    print(time)
    times.append(time)
    df_all = pd.read_csv(file)
    df = pd.merge(df_all, df_dhh1['Name'], how='inner', on=['Name'])
    df_dhh1 = df_dhh1[df_dhh1.Name.isin(df.Name)] 
    df = df[df.iloc[:,2] > 100]
    
    all_len = dict(zip(df['Name'],df['Length']))
    rbf_dict = dict(zip(df['Name'],df['PolarityPerGene']))
    print('In sample ', time, ' number of genes used ', len(df))
    pol_dict = {}
    for key in rbf_dict:
        if key in all_len:
            pol_dict.update({all_len.get(key): rbf_dict.get(key)})
        
    pol_dict_s = dict(sorted(pol_dict.items()))
    #print(pol_dict_s)

    agg_dict = dict()
    agg_pol = 0.0
    num_gene = 0
    for l in range(d1,viewer):
        if l in pol_dict_s.keys():
            #print(pol_dict_s.get(l))
            agg_pol += pol_dict_s.get(l)
            num_gene += 1
                
        if l % (num_c*3) == 0 and num_gene > 0:
            agg_dict.update({int(l/3):agg_pol/(num_c*3)})
            num_gene = 0
            agg_pol = 0
    
    #print(agg_dict)
    keys = np.array(list(agg_dict.keys()))
    values = np.array(list(agg_dict.values()))
    values_list.append(values)

-Glu15+Glu1-1_8_ACTTGA
In sample  -Glu15+Glu1-1_8_ACTTGA  number of genes used  4316
-Glu15+Glu5-2_9_ACGACT
In sample  -Glu15+Glu5-2_9_ACGACT  number of genes used  4314
-Glu15-1_7_CAGATC
In sample  -Glu15-1_7_CAGATC  number of genes used  4317
-Glu7_5-1_6_GCCAAT
In sample  -Glu7_5-1_6_GCCAAT  number of genes used  4315
logphase-1_5_ACAGTG
In sample  logphase-1_5_ACAGTG  number of genes used  4317


In [68]:
fig = go.Figure()

for t in range(len(times)):
    fig.add_trace(go.Scatter(x=keys, y=values_list[t],
                    mode='markers+lines',
                    #mode='markers',
                    name= times[t]))
    
# Set title
fig.update_layout(
    title_text="Polarity for all genes",
    xaxis_title="Length of gene(in codon)",
    yaxis_title="Averaged polarity per gene"
)

fig.show()